In [1]:
import pandas as pd
import numpy as np

In [97]:
print(f"Starting datetime: {pd.to_datetime('today')}")

Starting datetime: 2022-07-19 19:36:36.662080


In [237]:
data = pd.read_csv("../data/event_data.csv")

In [217]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79742 entries, 0 to 79741
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          79742 non-null  object 
 1   event_date       79742 non-null  object 
 2   event_type       79742 non-null  object 
 3   purchase_amount  6207 non-null   float64
dtypes: float64(1), object(3)
memory usage: 2.4+ MB


In [218]:
data.head(7)

,user_id,event_date,event_type,purchase_amount
0,c40e6a,2019-07-29 00:02:15,registration,NaN
1,a2b682,2019-07-29 00:04:46,registration,NaN
2,9ac888,2019-07-29 00:13:22,registration,NaN
3,93ff22,2019-07-29 00:16:47,registration,NaN
4,65ef85,2019-07-29 00:19:23,registration,NaN
5,90852e,2019-07-29 00:21:16,registration,NaN
6,357151,2019-07-29 00:25:53,registration,NaN


2. Highlight user cohorts based on the week of registration in the application. The cohort
identifier should be the week ordinal (for example, the week from July 29 to August 4
should have identifier 31).



In [238]:
data.event_date = data.event_date.apply(pd.Timestamp)

In [239]:
data.event_date.describe()

/var/folders/qr/1htx77yx7hqb1c_22kq9l2_w0000gn/T/ipykernel_774/3455934772.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  data.event_date.describe()


count                   79742
unique                  78509
top       2019-08-29 17:13:38
freq                        3
first     2019-07-29 00:02:15
last      2019-09-01 23:59:54
Name: event_date, dtype: object

In [240]:
data["week_number"] = data.event_date.dt.week

/var/folders/qr/1htx77yx7hqb1c_22kq9l2_w0000gn/T/ipykernel_774/3812958975.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data["week_number"] = data.event_date.dt.week


In [241]:
table = data.groupby("week_number", as_index = False).user_id.count()


3. How many unique users in the cohort with ID 33?



In [242]:
table.query("week_number == 33").user_id.values[0]

18144

4. For each event, highlight the indicator lifetime - the weekly lifetime of the cohort. The
lifetime indicator is calculated based on the serial number of the week in which the event
is committed, relative to the week of registration. For example, an event committed on
August 3 by a user from a cohort of registrants at 31 weeks will be committed on the zero
week of lifetime, and an event committed by the same user on August 5 will be committed
on the first week of lifetime).



In [246]:
data["cohort_lifetime"] = data.groupby("user_id").week_number.rank(method = "dense") - 1
data["cohort_id"] = data.groupby("user_id").week_number.transform("min")

In [248]:
data.sample(7)

,user_id,event_date,event_type,purchase_amount,week_number,cohort_lifetime,cohort_id
10156,3be5b0,2019-08-07 16:36:38,simple_event,NaN,32,0.0,32
20277,14b5dd,2019-08-12 14:07:14,simple_event,NaN,33,1.0,32
50171,b7274f,2019-08-23 12:14:38,registration,NaN,34,0.0,34
9873,7f5766,2019-08-07 13:02:18,simple_event,NaN,32,1.0,31
2531,e4ec01,2019-08-02 05:24:28,simple_event,NaN,31,0.0,31
69479,930cb5,2019-08-29 18:30:38,simple_event,NaN,35,1.0,34
2696,4a0550,2019-08-02 09:40:45,simple_event,NaN,31,0.0,31


5. Build a summary table of changes in the Retention Rate for cohorts depending on lifetime.



In [268]:
table = (data
 .groupby(["cohort_id","cohort_lifetime"], as_index = False)
 .user_id
 .count()
). assign(retention_rate = (data
 .groupby(["cohort_id","cohort_lifetime"], as_index = False)
 .user_id
 .count()
 .groupby(["cohort_id"])
 .user_id
 .pct_change()
))

table

,cohort_id,cohort_lifetime,user_id,retention_rate
0,31,0.0,5758,NaN
1,31,1.0,7439,0.291942
2,31,2.0,4671,-0.372093
3,31,3.0,2414,-0.483194
4,31,4.0,838,-0.652858
5,32,0.0,5706,NaN
6,32,1.0,7452,0.305994
7,32,2.0,4724,-0.366076
8,32,3.0,2447,-0.482007
9,33,0.0,6021,NaN


6. What is the 3 week retention rate for a cohort with ID 32? Give the answer in percent,
rounded to 2 decimal places, inclusive.



In [272]:
round(table.query("cohort_id == 32 and cohort_lifetime == 3").retention_rate.values[0],2)

-0.48

7. Build a summary table of changes in the indicator ARPPU (Average Revenue Per Paying
User) for cohorts depending on lifetime.



In [294]:
amount_table = (data
 .groupby(["cohort_id","cohort_lifetime"], as_index = False)
 .purchase_amount
 .sum()
    )

paid_users_table = (data[data.event_type == "purchase"]
 .groupby(["cohort_id","cohort_lifetime"], as_index = False)
 .user_id
 .nunique()
 .rename(columns = {"user_id":"paid_users"})
    )

table = pd.merge(amount_table, paid_users_table, on = ["cohort_id", "cohort_lifetime"], how = "outer")

In [296]:
table["arppu"] = table["purchase_amount"]/table["paid_users"]

In [302]:
table = (table
.assign(arpu_change = table.groupby("cohort_id").arppu.pct_change())
       )
table

,cohort_id,cohort_lifetime,purchase_amount,paid_users,arppu,arpu_change
0,31,0.0,8890.0,292,30.445205,NaN
1,31,1.0,20540.0,580,35.413793,0.163198
2,31,2.0,12210.0,340,35.911765,0.014062
3,31,3.0,6120.0,185,33.081081,-0.078823
4,31,4.0,2010.0,64,31.406250,-0.050628
5,32,0.0,10850.0,329,32.978723,NaN
6,32,1.0,21050.0,601,35.024958,0.062047
7,32,2.0,12600.0,357,35.294118,0.007685
8,32,3.0,6260.0,188,33.297872,-0.056560
9,33,0.0,9790.0,295,33.186441,NaN


8. What is the 3-week ARPPU of a cohort with ID 31? Give the answer with a floating point
number, rounded to 2 decimal places, inclusive.



In [303]:
round(table.query("cohort_id == 31 and cohort_lifetime == 3").arppu.values[0],2)

33.08

9. What is the median time between user registration and first purchase? Give the answer
in seconds (!) As an integer.

In [336]:
registration_table = table[table.event_type == "registration"].reset_index(drop = True)
registration_table.rename(columns = {"event_date":"registration_datetime"}, inplace = True)
registration_table.drop("event_type", axis = 1, inplace = True)

purchase_table = table[table.event_type == "purchase"].reset_index(drop = True)
purchase_table.rename(columns = {"event_date":"purchase_datetime"}, inplace = True)
purchase_table.drop("event_type", axis = 1, inplace = True)

In [337]:
merged = pd.merge(registration_table, purchase_table, on = ["user_id"], how = "outer")

In [357]:
t = (
merged[merged.purchase_datetime.notnull()]
.assign(time_between_purchs = lambda x: (x["purchase_datetime"] - x["registration_datetime"])/np.timedelta64(1,"s"))
).sort_values(by = "time_between_purchs")

In [360]:
int(t.time_between_purchs.median())

434774

In [361]:
print(f"End datetime: {pd.to_datetime('today')}")

End datetime: 2022-07-19 21:17:58.794796
